<a href="https://colab.research.google.com/github/james-trayford/AudibleUniverseWorkbooks/blob/group4/STRAUSSdemo_bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook prepared by **Dr James Trayford** - for queries please email [`james.trayford@port.ac.uk`](mailto:james.trayford@port.ac.uk)

To use this notebook (if you haven't already) you can first save a copy to your local drive by clicking `File > Save a Copy in Drive` and run on that copy. `Edit > Clear all outputs` on that copy should also ensure yopu have a clean version
 to start from.
 
 ***This is a bonus notebook for main activity [here](https://github.com/james-trayford/AudibleUniverseWorkbooks/blob/group4/STRAUSSdemo.ipynb), demonstrating the fundamentals in more detail.***

## **1. Setup:**



***Note: you may not need to install `strauss` again if you have done so already! If so can skip running the following cell***

First, let's install `strauss`! Just run the code cell below. 

*We will use the `spectraliser` development branch for this notebook. Install can take a while - but you should only need to run it once!*


In [ ]:
 !pip --quiet install git+https://github.com/james-trayford/strauss.git@spectraliser -U

and also make a local copy of the repository

In [ ]:
 !git clone https://github.com/james-trayford/strauss.git

Make plots appear in-line by default

In [ ]:
%matplotlib inline

Import the modules we need...

In [ ]:
# strauss imports
from strauss.sonification import Sonification
from strauss.sources import Events, Objects
from strauss import channels
from strauss.score import Score
from strauss.generator import Sampler, Synthesizer, Spectralizer
from strauss import sources as Sources 
import strauss

# other useful modules
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.signal import savgol_filter
import urllib.request
import os
import zipfile
import glob
import yaml
import pandas as pd

# modules to display in-notebook
import IPython.display as ipd
from IPython.core.display import display, Markdown, Latex, Image

# set figures to be a decent size by default
import matplotlib
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)
matplotlib.rc('figure', **{'figsize':[14.0, 7.0]})

## ***Bonus: Multivariate time-series***

Grab the bonus data...

In [ ]:
outdir = "./AU2_Group4"

path = os.path.realpath(outdir)
if not glob.glob(outdir): 
  os.mkdir(path)  
    
fname = "Group4_input_data_bonus.zip"
url = "https://drive.google.com/uc?export=download&id=1a6AHrfjsnYEzfLRMySsEN8HEttKAVkBg"

print(f"Downloading files...")
with urllib.request.urlopen(url) as response, open(f"{path}/{fname}", 'wb') as out_file:
  data = response.read() # a `bytes` object
  out_file.write(data)

print(f"Unzipping files to {outdir}/Data_bonus ...")
with zipfile.ZipFile(f"{outdir}/{fname}", 'r') as zip_ref:
    zip_ref.extractall(f"{outdir}")

print(f"Clearing up...")
os.remove(f"{path}/{fname}")

print("Done.")

Lets look at these multivariate data sets. These are star-formation and metal-enrichment histories of 5 galaxies from the [EAGLE simulations](https://eagle.strw.leidenuniv.nl/) (virtual galaxies, modelled on a supercomputer) 

We see that we have two time-dependent quantities: The rate at which stars form (aka ***SFR***, in solar masses per year) and the mass fraction of those stars in elements heavier than Helium (aka ***Metallicity*** or $Z$) 

Let's take a look...

In [ ]:
for f in glob.glob('./AU2_Group4/Data_bonus/*.txt'):
  data_table = pd.read_csv(f)
  display(Markdown(f"### File: <mark>`{f}`</mark>"))
  xlab = data_table.columns[0]
  ylab = data_table.columns[1]
  zlab = data_table.columns[2]
  plt.plot(data_table[xlab], data_table[ylab])
  plt.xlabel(f'{xlab}')
  plt.ylabel(f'{ylab}')
  plt.show()
  plt.plot(data_table[xlab], data_table[zlab])
  plt.xlabel(f'{xlab}')
  plt.ylabel(f'{zlab}')
  plt.show()
  print(data_table.head())
  #display(data_table)

First we will try sonifying these by mapping SFR to `pitch_shift` and Metallicity ($Z$) to low-pass filter `cutoff`  

In [ ]:
# some strauss setup can happen outside the loop...
generator = Synthesizer()
generator.load_preset('default')
generator.modify_preset({'filter':'on'})
notes = [["A2", "E3"]]
score =  Score(notes, length)

lims = {'time_evo': ('0','100'),
        'pitch_shift': ('0','100'),
        'cutoff': ('0','100')}

display(Markdown(f"## Mapping SFR to <mark>***`pitch_shift`***</mark> and Metallicity to <mark>***`cutoff`***</mark>:"))

for f in glob.glob('./AU2_Group4/Data_bonus/*.txt'):

  display(Markdown(f"### EAGLE Galaxy <mark>***`{f.split('.')[1].split('_')[-1]}`</mark>***:"))
  
  # get data
  data_table = np.genfromtxt(f, delimiter=',')[1:]
  time = data_table[:,0]
  sfr = data_table[:,1]
  Z = data_table[:,2]

  # plot multivariate data ------------------------ 
  plt.plot(time,sfr)
  plt.xlabel(f'Time [Gyr]')
  plt.ylabel(f'Star Formation Rate [Msun/year]')
  plt.gca().twinx()
  plt.plot([0],[0], label='Star Formation Rate') # dummy plot to make legend work
  plt.plot(time,Z,c='C1',label="Stellar Metallicity")
  plt.xlabel(f'Time [Gyr]')
  plt.ylabel(f'Metallicity')
  plt.legend(frameon=0)
  plt.show()
  # ------------------------------------------------ 

  %matplotlib notebook

  data = {'pitch':[0,1],
          'time_evo':[time]*2,
          'pitch_shift':[sfr]*2,
          'cutoff':[Z]*2}



  # set up source
  sources = Objects(data.keys())
  sources.fromdict(data)
  plims = {'cutoff': (0.3,0.9)}
  sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

  soni = Sonification(score, sources, generator, system)
  soni.render()
  dobj = soni.notebook_display()

  %matplotlib inline

# change back in case cells are run out of order...
generator.load_preset('pitch_mapper')

Now let's choose some combination of evolvable parameters!

Now there are 2 properties being sonified at once (not including the time variable), there are 28 possible combinations of these mappings $\left(\frac{8!}{2!(8-2)!} = 28\right)$ 

Some combinations might work better than others...

In [ ]:
# A list of some 'evolvable' mappings
some_mappings = ["pitch_shift",
                 "cutoff",
                 "volume",
                 "phi",
                 "volume_lfo/amount", 
                 "volume_lfo/freq_shift",
                 "pitch_lfo/amount", 
                 "pitch_lfo/freq_shift"]

# !! change these (between 0 and 7) to select a different property to map... 
idx_sfr = 1
idx_Z = 5

chosen = [some_mappings[idx_Z],some_mappings[idx_sfr]]

# use a stereo system to allow 'phi' mapping (low pan left and high pan right)
system = "stereo"

# some strauss setup can happen outside the loop...
generator = Synthesizer()

generator = Synthesizer()
generator.modify_preset({'filter':'on',
                         "pitch_hi":-1, "pitch_lo": 1,
                         "pitch_lfo": {"use": "on", 
                                       "amount":1*("pitch_lfo/freq_shift" in chosen), 
                                       "freq":3*5**("pitch_lfo/freq_shift" not in chosen), 
                                       "phase":0.25},
                         "volume_lfo": {"use": "on", 
                                        "amount":1*("volume_lfo/freq_shift" in chosen), 
                                        "freq":3*5**("volume_lfo/freq_shift" not in chosen), 
                                        "phase":0}
                        })


notes = [["A2", "E3"]]
score =  Score(notes, length)

lims = {'time_evo': ('0','100'),
        'pitch_shift': ('0','100'),
        'cutoff': ('0','100'),
        "volume": ('0','100'), 
        "phi": (-0.5,1.5),
        "volume_lfo/amount": ('0','100'), 
        "volume_lfo/freq_shift": ('0','100'),
        "pitch_lfo/amount": ('0','100'), 
        "pitch_lfo/freq_shift": ('0','100')}

display(Markdown(f"## Mapping SFR to <mark>***`{some_mappings[idx_sfr]}`***</mark> and Metallicity to <mark>***`{some_mappings[idx_Z]}`***</mark>:"))

for f in glob.glob('./AU2_Group4/Data_bonus/*.txt'):

  display(Markdown(f"### EAGLE Galaxy <mark>***`{f.split('.')[1].split('_')[-1]}`</mark>***:"))

  # get data
  data_table = np.genfromtxt(f, delimiter=',')[1:]
  time = data_table[:,0]
  sfr = data_table[:,1]
  Z = data_table[:,2]

  # plot multivariate data ------------------------ 
  plt.plot(time,sfr)
  plt.xlabel(f'Time [Gyr]')
  plt.ylabel(f'Star Formation Rate [Msun/year]')
  plt.gca().twinx()
  plt.plot([0],[0], label='Star Formation Rate') # dummy plot to make legend work
  plt.plot(time,Z,c='C1',label="Stellar Metallicity")
  plt.xlabel(f'Time [Gyr]')
  plt.ylabel(f'Metallicity')
  plt.legend(frameon=0)
  plt.show()
  # ------------------------------------------------ 

  %matplotlib notebook

  data = {'pitch':[0,1],
          'time_evo':[time]*2,
          'theta':[0.5]*2,
          some_mappings[idx_sfr]:[(sfr - sfr.min())/(sfr.max()-sfr.min())]*2,
          some_mappings[idx_Z]:[(Z - Z.min())/(Z.max()-Z.min())]*2}

  # set up source
  sources = Objects(data.keys())
  sources.fromdict(data)
  plims = {'cutoff': (0.4,1)}
  sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

  soni = Sonification(score, sources, generator, system)
  soni.render()
  dobj = soni.notebook_display()

  %matplotlib inline

# change back in case cells are run out of order...
generator.load_preset('pitch_mapper')